# 2. Global optimization and metaheuristic algortihms

Installing Pygmo 2.19.0 specifically for the reason that documentation samples don't compile for latest version available. Uhhhh~~~

In [2]:
!pip install pygmo==2.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 3.1 MB/s eta 0:00:00


In [47]:
from dataclasses import dataclass
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt
import pygmo
import pandas as pd
from IPython.display import display
%matplotlib inline

In [18]:
@dataclass
class Benchmark:
  name: str
  f: Callable[[float, float], float] # Formula
  gf: Callable[[float, float], np.array] # Gradient
  minimum: tuple # Global minimum point
  domain: tuple # Search domain per each component

  def __post_init__(self):
    self.dim = len(self.domain)

  def fitness(self, X):
    assert len(X) == self.dim
    x, y = X
    return [self.f(x, y)]

  def get_bounds(self):
    return ([it[0] for it in mccormick.domain],
            [it[1] for it in mccormick.domain])

  def get_name(self) -> str:
    return self.name

In [19]:
mccormick = Benchmark(
  name='McCormick function',
  f=lambda x, y: np.sin(x + y) + (x - y) ** 2 - 1.5 * x + 2.5 * y + 1,
  gf=lambda x, y: np.array([
    np.cos(x + y) + 2 * (x - y) - 1.5,
    np.cos(x + y) - 2 * (x - y) + 2.5
  ]),
  minimum=(-0.54719, -1.54719),
  domain=((-1.5, 4), (-3, 4))
)

In [20]:
booth = Benchmark(
  name='Booth function',
  f=lambda x, y: (x + 2 * y - 7) ** 2 + (2 * x + y - 5) ** 2,
  gf=lambda x, y: np.array([
    2 * (x + 2 * y - 7) + 4 * (2 * x + y - 5),
    4 * (x + 2 * y - 7) + 2 * (2 * x + y - 5)
  ]),
  minimum=(1, 3),
  domain=((-10, 10), (-10, 10))
)

In [21]:
benchmarks = [mccormick, booth]

Declare multiagent algorithms

In [22]:
GENERATIONS = 100
SIZE = 100

grey_wolves = pygmo.algorithm(pygmo.gwo(gen=GENERATIONS))
bee_colony = pygmo.algorithm(pygmo.bee_colony(gen=GENERATIONS, limit=GENERATIONS))
cmaes = pygmo.algorithm(pygmo.cmaes(gen=GENERATIONS))

algorithms = [grey_wolves, bee_colony, cmaes]

In [57]:
for bench in benchmarks:
  problem = pygmo.problem(bench)
  population = pygmo.population(problem, size=SIZE)

  best_xy = []
  best_z = []
  for algo in algorithms:
    pop = algo.evolve(population)
    best_xy.append(pop.champion_x)
    best_z.append(pop.champion_f[0])

  error = np.abs(np.array(best_z) - bench.f(*bench.minimum))
  table = pd.DataFrame({'Algorithm': [a.get_name().split(':')[0] for a in algorithms],
                        'Solution point': best_xy,
                        'Solution value': best_z,
                        'Absolute error': error})
  display(table.style.set_caption(bench.name))

,Algorithm,Solution point,Solution value,Absolute error
0,GWO,[-0.54712133 -1.54730378],-1.913223,0.000000
1,ABC,[-0.54719757 -1.54719757],-1.913223,0.000000
2,CMA-ES,[-8.72638616e+19 -8.72638616e+19],-85413985886930534400.000000,85413985886930534400.000000


,Algorithm,Solution point,Solution value,Absolute error
0,GWO,[1.00002783 3.00002184],0.000000,0.000000
1,ABC,[0.99988923 3.00012173],0.000000,0.000000
2,CMA-ES,[1.0000053 3.00000219],0.000000,0.000000
